# Long Sentence Resolution - Preprocess Shortening Datasets

# Documentation

## Datasets

Dataset | #examples before preprocessing | #examples after preprocessing | %examples kept after preprocessing
:- | :- | :- | :-
QQP | 149263 | 7634 | 5.1%
MRPC | 3900 | 624 | 16.0%
PAWS | 31834 | 1588 | 5.0%
Google sentence compression dataset | 210000 | 170473 | 81.2%
Gigaword | 3995559 | 2783868 | 69.7%
DUC 2004 | 2000 | 1648 | 82.4%

## What is a word?

* A word is any token that is not any of the following:
  * bracket
  * punctuation
  * quote
  * space
  * currency

## Connect Google Drive
 
* Run the cell "Connect Google Drive" and on the popup window click "Connect to Google Drive".
 
* In the new page opened, choose an account (your account) and click Allow.

## Configuration parameters

* **Source minimum #words**: The number of words that a sentence should have to be considered a valid source sentence. All sentences that have less words than this threshold are dropped (default: 17).

* **Target maximum #words**: The number of words that a sentence should have to be considered a valid target sentence. All sentences that have more words than this threshold are dropped (default: 16).

* **Minimum compression rate**: The minimum compression rate an example can have. All examples that have less than this threshold are dropped (default: 0).

# Installations

In [ ]:
!pip install datasets==2.5.2 \
             spacy==3.4.1

# Imports

In [ ]:
import datetime, spacy, json

from typing import List, Union, Dict, Iterator

from pathlib import Path

from datasets import Dataset, concatenate_datasets, load_from_disk

from google.colab import drive

from spacy.lang.en import English

from spacy.tokens import Doc, Token

import pandas as pd

import ipywidgets as widgets

import plotly.express as px

import numpy as np

# Configuration parameters

In [ ]:
source_minimum_n_words_bounded_int_text = widgets.BoundedIntText(value=17, min=1, max=20, step=1)

target_maximum_n_words_bounded_int_text = widgets.BoundedIntText(value=16, min=1, max=20, step=1)

minimum_compression_rate_bounded_float_text = widgets.BoundedFloatText(value=0.0, min=0.0, max=1.0, step=0.05)

display(widgets.HBox([widgets.Label(value="Source minimum #words:"), source_minimum_n_words_bounded_int_text]))

display(widgets.HBox([widgets.Label(value="Target maximum #words:"), target_maximum_n_words_bounded_int_text]))

display(widgets.HBox([widgets.Label(value="Minimum compression rate:"), minimum_compression_rate_bounded_float_text]))

# Connect Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Create paths

In [ ]:
RAW_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/raw')

PREPROCESSED_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/preprocessed')

PREPROCESSED_DATA_DIRECTORY_PATH.mkdir(parents=True, exist_ok=True)

# Functions / Classes

In [ ]:
def preprocess_qqp_dataset(ds: Dataset) -> Dataset:
  def map_operation(example: Dict) -> Dict:
    example["s1"] = example['s1'].strip()

    example["s2"] = example['s2'].strip()

    example["dataset"] = "QQP"

    example["full_path"] = create_full_path_from_dataset_example(example)

    return example

  return ds.filter(lambda o: o['label'] == 1) \
           .rename_columns({'question1': 's1', 'question2': 's2'}) \
           .map(map_operation)

def preprocess_mrpc_dataset(ds: Dataset) -> Dataset:
  def map_operation(example: Dict) -> Dict:
    example["s1"] = example['s1'].strip()

    example["s2"] = example['s2'].strip()

    example["dataset"] = "MRPC"

    example["full_path"] = create_full_path_from_dataset_example(example)

    return example

  return ds.filter(lambda o: o['label'] == 1) \
           .rename_columns({'sentence1': 's1', 'sentence2': 's2'}) \
           .map(map_operation)

def preprocess_paws_dataset(ds: Dataset) -> Dataset:
  def map_operation(example: Dict) -> Dict:
    example["s1"] = example['s1'].strip()

    example["s2"] = example['s2'].strip()

    example["dataset"] = "PAWS"

    example["full_path"] = create_full_path_from_dataset_example(example)

    return example

  return ds.filter(lambda o: o['label'] == 1) \
           .rename_columns({'sentence1': 's1', 'sentence2': 's2'}) \
           .map(map_operation)

def preprocess_google_sentence_compression_dataset(ds: Dataset) -> Dataset:
  def map_operation(example: Dict) -> Dict:
    example["s1"] = example["graph"]['sentence'].strip()

    example["s2"] = example['compression']['text'].strip()

    example["dataset"] = "Google sentence compression dataset"

    example["full_path"] = create_full_path_from_dataset_example(example)

    return example

  return ds.map(map_operation)

def preprocess_gigaword_dataset(ds: Dataset) -> Dataset:
  def map_operation(example: Dict) -> Dict:
    example["s1"] = example['s1'].strip()

    example["s2"] = example['s2'].strip()

    example["dataset"] = "Gigaword"

    example["full_path"] = create_full_path_from_dataset_example(example)

    return example

  return ds.rename_columns({'document': 's1', 'summary': 's2'}).map(map_operation)

def preprocess_duc2004_dataset(ds: Dataset) -> Dataset:
  def map_operation(example: Dict) -> Dict:
    example["s1"] = example['s1'].strip()

    example["s2"] = example['s2'].strip()

    example["dataset"] = "DUC 2004"

    example["full_path"] = create_full_path_from_dataset_example(example)

    return example

  temp = ds.to_pandas().explode('summaries', ignore_index=True)

  return Dataset.from_pandas(temp, preserve_index=False) \
    .rename_columns({'text': 's1', 'summaries': 's2'})   \
    .map(map_operation)

class DocumentAnalysis:
  def __init__ (self):
    self._linguistics_model = English()

    self._linguistics_model.add_pipe("sentencizer")

  def analyze_documents(self, documents: Union[List[str], np.ndarray, pd.Series]) -> Iterator[Doc]:
    return self._linguistics_model.pipe(documents, batch_size=50)

  def extract_n_words_from_documents(self, documents: Union[List[str], np.ndarray, pd.Series]) -> np.ndarray:
    return np.array(list(map(self.extract_n_words_from_document, self.analyze_documents(documents))))

  def extract_n_words_from_document(self, document: Doc) -> int:
    return sum(map(self._token_is_word, document))

  def extract_n_sentences_from_document(self, document: Doc) -> int:
    return len(list(document.sents))

  def _token_is_word(self, token: Token) -> bool:
    return not any([ token.is_bracket,
                     token.is_punct,
                     token.is_currency,
                     token.is_quote,
                     token.is_space ]) # https://spacy.io/api/token#attributes

class WordBasedCompressionRate:
  def __init__ (self):
    self._document_analysis = DocumentAnalysis()

  def compute(self, uncompressed_documents: Union[List[str], np.ndarray, pd.Series], compressed_documents: Union[List[str], np.ndarray, pd.Series], use_aggregator: bool = True) -> Union[Dict, float]:
    assert(len(uncompressed_documents) == len(compressed_documents))

    uncompressed_n_words = self._document_analysis.extract_n_words_from_documents(uncompressed_documents)

    compressed_n_words = self._document_analysis.extract_n_words_from_documents(compressed_documents)

    compression_rates = compressed_n_words / uncompressed_n_words

    return np.mean(compression_rates) if use_aggregator else {
        'compression_rates': compression_rates,
        'uncompressed_n_words': uncompressed_n_words,
        'compressed_n_words': compressed_n_words
    }

def create_full_path_from_dataset_example(example: Dict) -> str:
  full_path = example["dataset"]

  if 'subset' in example:
    full_path = full_path + f'>{example["subset"]}'

  if 'split' in example:
    full_path = full_path + f'>{example["split"]}'

  return full_path

# Load raw datasets

In [ ]:
%%time

qqp_dataset = load_from_disk(RAW_DATA_DIRECTORY_PATH / 'qqp')

qqp_dataset

In [ ]:
%%time

mrpc_dataset = load_from_disk(RAW_DATA_DIRECTORY_PATH / 'mrpc')

mrpc_dataset

In [ ]:
%%time

paws_dataset = load_from_disk(RAW_DATA_DIRECTORY_PATH / 'paws')

paws_dataset

In [ ]:
%%time

google_sentence_compression_dataset = load_from_disk(RAW_DATA_DIRECTORY_PATH / 'google_sentence_compression_dataset')

google_sentence_compression_dataset

In [ ]:
%%time

gigaword_dataset = load_from_disk(RAW_DATA_DIRECTORY_PATH / 'gigaword')

gigaword_dataset

In [ ]:
%%time

duc2004_dataset = load_from_disk(RAW_DATA_DIRECTORY_PATH / 'duc2004')

duc2004_dataset

# Preprocess raw datasets

## Normalize raw datasets

In [ ]:
%%time

qqp_dataset = preprocess_qqp_dataset(qqp_dataset)

qqp_dataset

In [ ]:
%%time

mrpc_dataset = preprocess_mrpc_dataset(mrpc_dataset)

mrpc_dataset

In [ ]:
%%time

paws_dataset = preprocess_paws_dataset(paws_dataset)

paws_dataset

In [ ]:
%%time

google_sentence_compression_dataset = preprocess_google_sentence_compression_dataset(google_sentence_compression_dataset)

google_sentence_compression_dataset

In [ ]:
%%time

gigaword_dataset = preprocess_gigaword_dataset(gigaword_dataset)

gigaword_dataset

In [ ]:
%%time

duc2004_dataset = preprocess_duc2004_dataset(duc2004_dataset)

duc2004_dataset

## Merge raw datasets

In [ ]:
shortening_datasets_to_concatenate = [ ds.remove_columns(set(ds.column_names).difference(['s1', 's2', 'dataset', 'split', 'subset', 'full_path'])) for ds in [
  qqp_dataset,
  mrpc_dataset,
  paws_dataset,
  google_sentence_compression_dataset,
  gigaword_dataset,
  duc2004_dataset
]]

shortening_dataset = concatenate_datasets(shortening_datasets_to_concatenate)

shortening_dataset

## Convert raw dataset to dataframe

In [ ]:
shortening_dataset_df = shortening_dataset.to_pandas()

In [ ]:
shortening_dataset_df

## Count #words and #sentences

In [ ]:
document_analysis = DocumentAnalysis()

In [ ]:
%%time

shortening_dataset_df[['s1_n_sentences', 's1_n_words']] = pd.DataFrame(map(lambda o: { 'n_sentences' : document_analysis.extract_n_sentences_from_document(o), 'n_words': document_analysis.extract_n_words_from_document(o) }, document_analysis.analyze_documents(shortening_dataset_df['s1'])), index=shortening_dataset_df.index)

shortening_dataset_df[['s2_n_sentences', 's2_n_words']] = pd.DataFrame(map(lambda o: { 'n_sentences' : document_analysis.extract_n_sentences_from_document(o), 'n_words': document_analysis.extract_n_words_from_document(o) }, document_analysis.analyze_documents(shortening_dataset_df['s2'])), index=shortening_dataset_df.index)

In [ ]:
shortening_dataset_df

## Statistics on raw datasets

### \#examples and %examples per dataset

In [ ]:
shortening_dataset_df['dataset'].value_counts()

In [ ]:
np.round((shortening_dataset_df['dataset'].value_counts(normalize=True) * 100), 2).astype(str) + '%'

### 5-number summaries for #words and #sentences per dataset

In [ ]:
shortening_dataset_df.dtypes

In [ ]:
shortening_dataset_df.groupby('dataset') [['s1_n_words', 's2_n_words']].describe().apply(lambda s: s.apply('{0:.5f}'.format)).T

In [ ]:
shortening_dataset_df.groupby('dataset') [['s1_n_sentences', 's2_n_sentences']].describe().apply(lambda s: s.apply('{0:.5f}'.format)).T

### Box plots for #words per dataset (20% sampled)

In [ ]:
px.box(shortening_dataset_df.groupby('dataset', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42)), x="dataset", y="s1_n_words")

In [ ]:
px.box(shortening_dataset_df.groupby('dataset', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42)), x="dataset", y="s2_n_words")

## Re-arrange source-target pairs

In [ ]:
rearrange_source_target_mask = shortening_dataset_df['s1_n_words'] < shortening_dataset_df['s2_n_words']

shortening_dataset_df.loc[rearrange_source_target_mask, ['s1', 's1_n_sentences', 's1_n_words', 's2', 's2_n_sentences', 's2_n_words']] = (shortening_dataset_df.loc[rearrange_source_target_mask, ['s2', 's2_n_sentences', 's2_n_words', 's1', 's1_n_sentences', 's1_n_words']].values)

In [ ]:
shortening_dataset_df

## Measure compression rate

In [ ]:
word_based_compression_rate_metric = WordBasedCompressionRate()

In [ ]:
%%time

shortening_dataset_df['compression_rate'] = word_based_compression_rate_metric.compute(shortening_dataset_df['s1'], shortening_dataset_df['s2'], use_aggregator=False) ['compression_rates']

In [ ]:
shortening_dataset_df

## Filter source-target pairs

In [ ]:
shortening_dataset_df = shortening_dataset_df[(shortening_dataset_df['s1_n_sentences'] == 1) & (shortening_dataset_df['s2_n_sentences'] == 1)]

In [ ]:
shortening_dataset_df = shortening_dataset_df[(shortening_dataset_df['s1_n_words'] >= source_minimum_n_words_bounded_int_text.value) & (shortening_dataset_df['s2_n_words'] <= target_maximum_n_words_bounded_int_text.value)]

In [ ]:
shortening_dataset_df = shortening_dataset_df[shortening_dataset_df['compression_rate'] >= minimum_compression_rate_bounded_float_text.value]

In [ ]:
shortening_dataset_df.drop_duplicates(subset=['s1', 's2'], inplace=True, ignore_index=True)

In [ ]:
shortening_dataset_df

## Save preprocessed dataset

In [ ]:
preprocessed_shortening_dataset_df = shortening_dataset_df.rename(columns={'s1': 'source', 's2': 'target'}) [['source', 'target', 'dataset', 'subset', 'split', 'full_path']]

preprocessed_shortening_dataset_df

In [ ]:
preprocessed_dataset_directory_path = PREPROCESSED_DATA_DIRECTORY_PATH / f'shortening_dataset_{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}'

In [ ]:
preprocessed_shortening_dataset = Dataset.from_pandas(preprocessed_shortening_dataset_df, preserve_index=False)

preprocessed_shortening_dataset.save_to_disk(preprocessed_dataset_directory_path)

with open(preprocessed_dataset_directory_path / 'config.json', 'w') as f: json.dump({
  "source_minimum_n_words" : source_minimum_n_words_bounded_int_text.value,
  "target_maximum_n_words" : target_maximum_n_words_bounded_int_text.value,
  "minimum_compression_rate" : minimum_compression_rate_bounded_float_text.value,
  "full_paths_in_preprocessed_data" : sorted(preprocessed_shortening_dataset.unique('full_path'))
}, f, sort_keys=True, indent=4)